In [2]:
search_for = 135
start_from = 0
threads = 10
thread = 7
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.016522884368896484
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 175
all cases: 723027151
type: [1, 1, 1, 1, 135] 135
cases of this type: 332150625
100000 32.61282362739111
200000 51.55799231356823
300000 54.43409459886771
400000 56.12700548667559
500000 65.27469175978905
600000 84.32314644717478
700000 80.24167122554205
800000 79.59014841403486
900000 76.94494933873659
1000000 71.96008911758192
1100000 75.4592398065974
1200000 62.374446021698965
1300000 29.437820055802394
1400000 72.36594835922936
1500000 71.3778038749058
1600000 70.50035222050178
1700000 71.45985341252387
1800000 75.52825548787246
1900000 73.95258458497143
2000000 71.06326856493153
2100000 70.11920738621333
2200000 73.29759250852217
2300000 70.17874826390496
2400000 70.47234271863412
2500000 22.527823344336916
2600000 65.69259363520554
2700000 84.77466525115881
2800000 83.23091950871658
2900000 85.3621671911064
3000000 83.00649793479545


30100000 76.25224219915043
30200000 72.51572735749879
30300000 71.7492957542351
30400000 75.73835866345651
30500000 70.62788341559468
30600000 73.22292101063164
30700000 70.79251483391289
30800000 15.505719265984396
30900000 72.76565886449399
31000000 72.36837464288621
31100000 75.31594982363836
31200000 73.29038437739521
31300000 73.46807178524044
31400000 75.1875178843895
31500000 77.48754822822326
31600000 75.98957643307598
31700000 73.64862716226563
31800000 26.76166458592233
31900000 78.0962399226355
32000000 73.53105139472552
32100000 77.68602171426151
32200000 70.0410658152594
32300000 31.323727403779866
32400000 75.00516978748739
32500000 73.0354720312595
32600000 72.98536263892514
32700000 69.34121480377271
32800000 75.29143633778351
32900000 73.0106906523579
33000000 68.9349210897245
33100000 68.22329491481308
33200000 53.738029664513626
33300000 38.69263315561301
33400000 69.6206537631877
33500000 72.44795621623119
33600000 71.67658481604468
33700000 75.89570895342206
338000

60600000 63.69450108642428
60700000 63.452426591647324
60800000 66.51382850047204
60900000 67.29103272560265
61000000 67.40869887760861
61100000 30.549033235272706
61200000 74.28525496148346
61300000 69.90611581596724
61400000 74.13342725686365
61500000 71.9000166642256
61600000 73.136216781918
61700000 71.16632035664438
61800000 68.00862610696868
61900000 76.0419921616958
62000000 36.626624222238206
62100000 60.73629151286764
62200000 67.26407334093288
62300000 65.69306869527826
62400000 68.74779412300875
62500000 64.78283494091843
62600000 62.08252834174545
62700000 59.641972145676775
62800000 19.80070123401867
62900000 62.982768169666905
63000000 62.984283207634164
63100000 63.2682627888815
63200000 62.687527166224775
63300000 66.44133629712297
63400000 66.42425026622199
63500000 46.51703394361716
63600000 49.892500786418935
63700000 69.76773954176501
63800000 70.48958944903215
63900000 70.26473299375006
64000000 70.01447385300588
64100000 71.32006303039107
64200000 71.0430079988010

90900000 69.40903918777957
91000000 61.46284514964977
91100000 65.51039473707381
91200000 66.3887443767895
91300000 66.10399919984859
91400000 65.68316918353007
91500000 61.36911940186421
91600000 62.261401461442674
91700000 48.07511690205149
91800000 35.791203081498125
91900000 58.06035344142278
92000000 59.298807049481525
92100000 59.676021120358826
92200000 57.5479205591253
92300000 19.83699844086234
92400000 61.11204566799715
92500000 58.05035816223458
92600000 56.96095900927355
92700000 56.756937794673505
92800000 45.05218911178459
92900000 38.51380765990907
93000000 63.97979475623552
93100000 62.04866965557532
93200000 64.98599498032898
93300000 63.21131753191233
93400000 62.715992651908735
93500000 66.4374104772758
93600000 69.19878518867108
93700000 67.03361481183228
93800000 65.45544793823359
93900000 66.71405196005863
94000000 62.80900108015815
94100000 63.004556253548365
94200000 38.13939513067229
94300000 40.64428505238831
94400000 60.641196840786506
94500000 61.77548588965

120400000 67.19896236659096
120500000 63.953994133729374
120600000 65.88586271397678
120700000 69.05763347847908
120800000 63.19216249808617
120900000 66.87064477995047
121000000 62.52931935258595
121100000 61.71530333899038
121200000 56.600354544653726
121300000 58.34635904421616
121400000 59.30388390517895
121500000 22.187426529072543
121600000 55.378563491578966
121700000 57.5957408147481
121800000 30.376943090686034
121900000 43.44680172942264
122000000 56.051400516217974
122100000 48.53354485124375
122200000 35.283116499765136
122300000 57.13315011511206
122400000 58.3985072980446
122500000 56.29203486809975
122600000 56.69490830158115
122700000 62.7402625233422
122800000 60.07989441446376
122900000 66.15466732465829
123000000 66.03053736189479
123100000 68.39846819717668
123200000 68.1100397384047
123300000 63.479542422681405
123400000 62.00282656390402
123500000 60.02213909529961
123600000 59.52126082817131
123700000 60.23946934208356
123800000 59.18762052456632
123900000 59.322

...

639700000 2.0049707003830193
639800000 1.7109039804439035
639900000 1.7266689119363776
640000000 1.6910940316226721
640100000 1.8306148873387431
640200000 1.8998790276284252
640300000 1.8750022725389384
640400000 1.7500974440317505
640500000 1.7055799353009236
640600000 1.6883579880931598
640700000 1.668013198705693
640800000 1.681891363330763
640900000 1.8045267467912318
641000000 2.0951939015270717
641100000 1.9006061610719664
641200000 1.6750344567401332
641300000 1.6016076181472283
641400000 1.609313544373603
641500000 1.5669802839027547
641600000 1.6599552801397668
641700000 1.7493238872336239
641800000 1.906449898903978
641900000 1.9875486149819837
642000000 1.7576296232272597
642100000 1.6196998989613016
642200000 1.5416796479079127
642300000 1.4802689671796752
642400000 1.495514889580699
642500000 1.5776048298041168
642600000 1.7298976034452946
642700000 1.8120606506898047
642800000 1.7721870482100084
642900000 1.6335331643223632
643000000 1.5688022987564447
643100000 1.4532098

667300000 2.273126740845692
667400000 2.4411732373822215
667500000 2.398142804703336
667600000 2.3964998023384987
667700000 2.4658399722859654
best so far: 0
type: [1, 1, 5, 3, 9] 135
cases of this type: 4428675
667800000 1.5921927153814017
667900000 0.9516846082896531
668000000 0.940209458360323
668100000 0.9525448633217771
668200000 0.9498812607861876
668300000 0.9576532626853598
668400000 0.9420260774055448
668500000 0.9419095672478119
668600000 0.9356567266715096
668700000 0.9526725230576197
668800000 0.9962454482563019
668900000 0.9559456698081799
669000000 0.9396810124161051
669100000 0.91332023955609
669200000 0.9550291556410684
669300000 1.0405089040779028
669400000 1.0557490336601139
669500000 0.9477453977564216
669600000 0.973145583966826
669700000 0.9648290993885039
669800000 1.2237335288004147
669900000 1.2820886542135737
670000000 1.1192026071824113
670100000 1.029521467455717
670200000 1.1352704327254766
670300000 1.2457161836564024
670400000 1.2069970718393723
670500000 

692000000 1.2416944473236708
692100000 2.018313914474277
692200000 1.6784193260009528
692300000 1.2111924998982277
692400000 2.332333371520432
692500000 1.455405081828922
692600000 1.33052228537474
692700000 2.290696357114477
692800000 1.154850117174061
692900000 1.5525973717460209
693000000 1.9984274644447022
693100000 1.447114183112508
693200000 1.7403718332897968
693300000 1.837808559565272
693400000 1.264185753330356
693500000 1.851236447498005
693600000 1.4356709260393115
693700000 1.3552111705354153
693800000 1.6662783672260615
693900000 1.4209945193181466
694000000 1.80986801773777
694100000 1.6301809029273173
694200000 1.4317037393387564
694300000 1.843114030318264
694400000 1.6008123925876478
694500000 1.5272769159960482
694600000 1.3575294932285626
694700000 1.5688267544391652
694800000 1.8800142550092158
694900000 1.462895146974212
695000000 1.497007024861602
695100000 1.6516768716990171
695200000 1.6049724004958632
695300000 1.4897017730714766
695400000 1.3273345797542209
6

713200000 0.10453392913587888
best so far: 0
type: [1, 9, 1, 15, 1] 135
cases of this type: 30375
best so far: 0
type: [1, 9, 3, 1, 5] 135
cases of this type: 50625
best so far: 0
type: [1, 9, 3, 5, 1] 135
cases of this type: 10125
713300000 0.09097600689556268
best so far: 0
type: [1, 9, 5, 1, 3] 135
cases of this type: 18225
best so far: 0
type: [1, 9, 5, 3, 1] 135
cases of this type: 6075
best so far: 0
type: [1, 9, 15, 1, 1] 135
cases of this type: 2025
best so far: 0
type: [1, 15, 1, 1, 9] 135
cases of this type: 98415
713400000 0.10539061833812859
best so far: 0
type: [1, 15, 1, 3, 3] 135
cases of this type: 32805
best so far: 0
type: [1, 15, 1, 9, 1] 135
cases of this type: 10935
best so far: 0
type: [1, 15, 3, 1, 3] 135
cases of this type: 10935
best so far: 0
type: [1, 15, 3, 3, 1] 135
cases of this type: 3645
best so far: 0
type: [1, 15, 9, 1, 1] 135
cases of this type: 1215
best so far: 0
type: [1, 27, 1, 1, 5] 135
cases of this type: 16875
713500000 0.10609593115985196
best

best so far: 0
type: [9, 1, 1, 3, 5] 135
cases of this type: 16875
best so far: 0
type: [9, 1, 1, 5, 3] 135
cases of this type: 10125
best so far: 0
type: [9, 1, 1, 15, 1] 135
cases of this type: 3375
723000000 0.0003335188935511642
best so far: 0
type: [9, 1, 3, 1, 5] 135
cases of this type: 5625
best so far: 0
type: [9, 1, 3, 5, 1] 135
cases of this type: 1125
best so far: 0
type: [9, 1, 5, 1, 3] 135
cases of this type: 2025
best so far: 0
type: [9, 1, 5, 3, 1] 135
cases of this type: 675
best so far: 0
type: [9, 1, 15, 1, 1] 135
cases of this type: 225
best so far: 0
type: [9, 3, 1, 1, 5] 135
cases of this type: 1875
best so far: 0
type: [9, 3, 1, 5, 1] 135
cases of this type: 375
best so far: 0
type: [9, 3, 5, 1, 1] 135
cases of this type: 75
best so far: 0
type: [9, 5, 1, 1, 3] 135
cases of this type: 405
best so far: 0
type: [9, 5, 1, 3, 1] 135
cases of this type: 135
best so far: 0
type: [9, 5, 3, 1, 1] 135
cases of this type: 45
best so far: 0
type: [9, 15, 1, 1, 1] 135
cases o